# Kaggle Competition:  Iceberg or Ship

## Notebook 3
## run resnext50 architecture

In [1]:
!ls -l

total 516
-rw-rw-r-- 1 nidhin nidhin 120934 Jan 13 18:20 1_getdata.ipynb
-rw-rw-r-- 1 nidhin nidhin 245611 Jan 13 17:48 2_model_resnet34.ipynb
-rw-rw-r-- 1 nidhin nidhin   7274 Jan 13 19:56 3_model_resnext50.ipynb
-rw-rw-r-- 1 nidhin nidhin 136525 Jan 13 20:16 baseline.ipynb
-rw-rw-r-- 1 nidhin nidhin    536 Jan 13 19:15 helper.py
drwxrwxr-x 2 nidhin nidhin   4096 Jan 13 19:15 __pycache__
-rw-rw-r-- 1 nidhin nidhin    185 Jan 13 17:48 README.md


In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *

In [4]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import pandas as pd
import numpy as np
import shutil
from IPython.display import FileLink, FileLinks

from sklearn.model_selection import train_test_split
import helper

In [6]:
#PATH = "/home/nidhin/data/iceberg/"
PATH = os.path.expanduser("~/data/iceberg/")

In [7]:
!ls -l {PATH}

total 1975496
drwxrwxr-x  2 nidhin nidhin       4096 Jan 13 19:06 models
-rw-rw-r--  1 nidhin nidhin     117951 Oct 23 17:27 sample_submission.csv
-rw-rw-r--  1 nidhin nidhin      38566 Jan  7 20:28 sample_submission.csv.7z
drwxrwxr-x  3 nidhin nidhin       4096 Jan 13 19:20 sub
drwxrwxr-x  2 nidhin nidhin     286720 Jan 13 17:51 test
-rw-rw-r--  1 nidhin nidhin 1521771850 Oct 23 17:27 test.json
-rw-rw-r--  1 nidhin nidhin  257127394 Jan  7 20:28 test.json.7z
drwxrwxr-x 11 nidhin nidhin       4096 Jan 13 17:59 tmp
drwxrwxr-x  4 nidhin nidhin       4096 Jan 13 17:50 train
-rw-rw-r--  1 nidhin nidhin  196313674 Oct 23 17:23 train.json
-rw-rw-r--  1 nidhin nidhin   44932785 Jan  7 20:28 train.json.7z
-rw-rw-r--  1 nidhin nidhin    2275916 Jan  8 01:44 train_modified.zip
drwxrwxr-x  4 nidhin nidhin       4096 Jan 13 17:50 valid


In [8]:
sz=299
arch=resnext50
bs=20

In [9]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs,test_name='test',)
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [10]:
learn.fit(1e-2, 8)
learn.precompute=False

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

[ 0.       0.7091   0.45644  0.78529]                      
[ 1.       0.6642   0.49435  0.78824]                       
[ 2.       0.5934   0.42044  0.79706]                       
[ 3.       0.58242  0.46514  0.78824]                       
[ 4.       0.53562  0.46626  0.81765]                       
[ 5.       0.49469  0.51166  0.80588]                       
[ 6.       0.46883  0.43528  0.80588]                       
[ 7.       0.4694   0.46571  0.81765]                       



In [11]:
learn.fit(1e-2, 4, cycle_len=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

[ 0.       0.53325  0.40753  0.81765]                      
[ 1.       0.49706  0.37624  0.81765]                      
[ 2.       0.45431  0.39204  0.80588]                      
[ 3.       0.3943   0.39091  0.82647]                      
[ 4.       0.42441  0.38083  0.80882]                      
[ 5.       0.39892  0.38844  0.82059]                      
[ 6.       0.40664  0.38123  0.81765]                      
[ 7.       0.37856  0.3846   0.83235]                      



In [12]:
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])

In [13]:
learn.fit(lr, 3, cycle_len=3,)

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[ 0.       0.64825  1.3963   0.55   ]                      
[ 1.       0.50004  0.62465  0.67353]                      
[ 2.       0.38986  0.56299  0.68824]                      
[ 3.       0.38554  0.41924  0.78824]                      
[ 4.       0.34081  0.56143  0.70588]                      
[ 5.       0.31654  0.54977  0.69412]                      
[ 6.       0.33152  1.01832  0.51765]                      
[ 7.       0.29405  0.67244  0.57353]                      
[ 8.       0.21958  0.75355  0.49706]                      



In [14]:
learn.save('224_all_50')

In [15]:
learn.load('224_all_50')

In [16]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),axis=0)
(metrics.log_loss(y,preds), accuracy(preds,y))

(0.31361453597813937, 0.88161993769470404)

In [23]:
log_preds,y = learn.TTA(is_test=True)

In [24]:
test_df = pd.read_json(PATH+"/test.json")
helper.make_test_file(log_preds,test_df,model_name='resnext50_sz299_bs20',
                      classes=data.classes,
                      submission_folder =f'{PATH}sub'
                     )

Saving to /home/nidhin/data/iceberg/sub/resnext50_sz299_bs20subm.gz
